In [3]:
import caffe
import lmdb
from PIL import Image

in_db = lmdb.open('image-lmdb', map_size=int(1e12))
with in_db.begin(write=True) as in_txn:
    for in_idx, in_ in enumerate(inputs):
        # load image:
        # - as np.uint8 {0, ..., 255}
        # - in BGR (switch from RGB)
        # - in Channel x Height x Width order (switch from H x W x C)
        im = np.array(Image.open(in_)) # or load whatever ndarray you need
        im = im[:,:,::-1]
        im = im.transpose((2,0,1))
        im_dat = caffe.io.array_to_datum(im)
        in_txn.put('{:0>10d}'.format(in_idx), im_dat.SerializeToString())
in_db.close()

NameError: name 'inputs' is not defined

In [13]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

from sklearn.decomposition import PCA as sklearnPCA

# Load image dataset#
Dataset = 'mit' # 'twin', 'chicago' or 'mit', 'zhihu' or 'funnyFace'
if Dataset == 'twin':
    imPath = '../../processing/imageProcessing/paddedImages/'
    ext = '.png'
elif Dataset == 'chicago':
    imPath = '../../ChicagoFaceDataset/CFD Version 2.0/CFD 2.0 Images/'
    ext = 'N.jpg'
elif Dataset == 'mit':
    imPath = '../../MIT2kFaceDataset/2kfaces/'
    ext = '.jpg'
elif Dataset =='funnyFace':
    imPath = '../funnyFace/'
    ext = '.png'
else:
    imPath = '../../../zhihu/'
    ext = '.jpg'
imList = []
for dirpath, dirnames, filenames in os.walk(imPath):
    for filename in [f for f in filenames if f.endswith(ext)]:
        imList.append(os.path.join(dirpath, filename))
imList.sort()
print len(imList)

import pandas as pd
ratingPath = '../Result/'+Dataset+'/meanRating.csv'
mean_rating = pd.read_csv(ratingPath,index_col = 0).as_matrix()[:,0].tolist()
mean_rating = map(float, mean_rating)
mean_rating = np.array(mean_rating)

import random
rand_index = range(len(imList))
random.shuffle(rand_index)

random_imList = [imList[i] for i in rand_index]
random_rating = mean_rating[rand_index]

split = 0.8
trainLen = int(len(imList)*split)
print trainLen
trainIm = random_imList[:trainLen]
testIm = random_imList[trainLen:]

trainRating = random_rating[:trainLen]
testRating = random_rating[trainLen:]

savePath = '../list/'
if not os.path.exists(savePath):
    os.makedirs(savePath)
f_train = open(savePath+'attr_train.txt','w')
f_test = open(savePath+'attr_test.txt','w')
for i in range(len(trainRating)):
    if i in range(len(testRating)):
        f_test.write(testIm[i]+' '+str(testRating[i])+'\n')
    f_train.write(trainIm[i]+' '+str(trainRating[i])+'\n')
    
f_train.close()
f_test.close()

2222
1777
